In [1]:
import keras
from keras import layers
import numpy  as np
import pandas as pd
from string import punctuation
import re
import random
import sys
import os

# LSTM Text Generation

Using section 8.1 in Deep Learning with Python as a guide, implement an LSTM text generator. Train the model on the Enron corpus or a text source of your choice. Save the model and generate 20 examples to the results directory of dsc650/assignments/assignment11/.

## Loading Recipe Data

In [2]:
df = pd.read_csv("Food Ingredients and Recipe Dataset with Image Name Mapping.csv")
df = df.sample(1000)
df

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
12298,12298,"Baked Halibut with Orzo, Spinach, and Cherry T...","['4 tablespoons extra-virgin olive oil, divide...",Preheat oven to 425°F. Whisk 2 tablespoons oil...,baked-halibut-with-orzo-spinach-and-cherry-tom...,"['4 tablespoons extra-virgin olive oil, divide..."
7968,7968,Warm Herbed Coriander Rice Salad,"['1 cup long-grain brown basmati rice, rinsed'...",Bring rice and water to a boil with 1 tablespo...,warm-herbed-coriander-rice-salad-358018,"['1 cup long-grain brown basmati rice, rinsed'..."
2534,2534,Crispy Rice Cakes With Tarator Sauce,"['Almond Aioli', '2 tablespoons fresh lemon ju...","Blend Almond Aioli, lemon juice, and garlic in...",crispy-rice-cakes-with-tarator-sauce,"['Almond Aioli', '2 tablespoons fresh lemon ju..."
10236,10236,Mexican Tuna Tostadas,"['8 corn tortillas (6 inches each)', '3 tables...",Heat oven to 350°F. Cut 2 rounds from each tor...,mexican-tuna-tostadas-241132,"['8 corn tortillas (6 inches each)', '3 tables..."
5944,5944,The House Salad,"['1 large head butter or bibb lettuce', '1 sma...","To make the dressing, whisk together the crème...",the-house-salad-51120200,"['1 large head butter or bibb lettuce', '1 sma..."
...,...,...,...,...,...,...
11409,11409,Carrot with Toasted Almond Soup,"['1 cup sliced shallots (about 4 large)', '1 T...","Cook shallots, bay leaf, ginger, curry powder,...",carrot-with-toasted-almond-soup-236394,"['1 cup sliced shallots (about 4 large)', '1 T..."
10971,10971,Mashed Turnips and Potatoes with Horseradish B...,['2 pounds yellow-fleshed potatoes such as Yuk...,"Peel potatoes and turnips, then cut into 2-inc...",mashed-turnips-and-potatoes-with-horseradish-b...,['2 pounds yellow-fleshed potatoes such as Yuk...
5298,5298,Creamy Green Gazpacho,"['1 medium tomato, cored and cut into quarters...","Reserve one-quarter of the tomato, two cucumbe...",creamy-green-gazpacho-51185630,"['1 medium tomato, cored and cut into quarters..."
7030,7030,Polenta Stack With Navy Bean Salad,"['Vegetable oil cooking spray', '2 cans (15.5 ...",Heat grill. Fold four 18-inch-long pieces of f...,polenta-stack-with-navy-bean-salad-365850,"['Vegetable oil cooking spray', '2 cans (15.5 ..."


## Data Cleaning and Structuring

In [3]:
df['Cleaned_Ingredients'] = df['Cleaned_Ingredients'].str.strip("[]")
df['Cleaned_Ingredients'] = df['Cleaned_Ingredients'].str.replace("'", '')
cleaned_ingredients = [i for i in df['Cleaned_Ingredients'].str.replace(",","\n")]

In [4]:
recipes = [f"{title}\n\n{ingredients}\n\n{instructions}"for title, ingredients, instructions in zip(df['Title'], cleaned_ingredients, df['Instructions'])]
text = "\n\n".join(i for i in recipes)

print(recipes[0])

Baked Halibut with Orzo, Spinach, and Cherry Tomatoes

4 tablespoons extra-virgin olive oil
 divided
 2 tablespoons fresh lemon juice
 2 6- to 7-ounce halibut fillets
 1 cup orzo (rice-shaped pasta)
 1 garlic clove
 minced
 4 cups (packed) baby spinach
 1 cup halved cherry tomatoes

Preheat oven to 425°F. Whisk 2 tablespoons oil and lemon juice in bowl; season dressing with salt and pepper. Place halibut on rimmed baking sheet; sprinkle with salt and pepper. Drizzle with some of dressing. Bake until just opaque in center, about 12 minutes.
Meanwhile, cook pasta in large saucepan of boiling salted water until tender but still firm to bite; drain. Add 2 tablespoons oil and garlic to same saucepan; sauté over medium heat 1 minute. Add drained pasta, spinach, and tomatoes; stir to coat. Season with salt and pepper. Remove from heat. Cover; let stand 1 minute (spinach will wilt). Divide pasta between 2 plates. Top with halibut and remaining dressing.


## Building the Input

In [5]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [6]:
print(f'Corpus Length: {len(text)}')
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print(chars)

Corpus Length: 1443576
Number of sequences: 481172
Unique characters: 134
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '\x81', '\x96', '¡', '®', '°', 'º', '¼', '½', '¾', '×', 'à', 'á', 'â', 'ç', 'è', 'é', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ù', 'ú', 'û', 'ü', 'ō', 'ź', '˚', '‑', '–', '—', '‘', '’', '“', '”', '⁄', '⅓', '⅔', '⅛', '◊', '�']


## LSTM Model

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



for epoch in range(1, 20):
    print('\nEpoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    model.save(f'Recipe_Generator_{epoch}.h5')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    
    print('--- Generating with seed: "' + generated_text + '"')

    with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
        writer.write(f'GENERATED TEXT FOR EPOCH {epoch}')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n------_Temperature:', temperature)

        with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
            writer.write('\n-------------------\nTEMPERATURE: ' + str(temperature) + '\n')
            writer.write((generated_text).upper() + '\n\n')

            sys.stdout.write(generated_text.upper())

            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                generated_text += next_char
                generated_text = generated_text[1:]

                sys.stdout.write(next_char)
                writer.write(next_char)

        print(generated_text)


model.save('Recipe_Generator.h5')

Vectorization...

Epoch 1
3760/3760 [==============================] - 467s 117ms/step - loss: 1.8697
--- Generating with seed: "to form a rough ball. Knead a few times to combine, then rol"

------_Temperature: 0.2
TO FORM A ROUGH BALL. KNEAD A FEW TIMES TO COMBINE, THEN ROLl to batches in a small bowl. Add batches and season with salt and pepper. Add batches and sugar, and sugar, and salt and pepper. Remove from heat and season with salt and pepper. Stir in cooked sauce and cut into broth and serve.

Cooked seeds
 1 teaspoon salt
 1 teaspoon salt
 1/2 cup sugar
 2 tablespoons chopped
 1 teaspoon salt
 1/2 cup chopped
 1 teaspoon freshly ground pepper
 1 teaspoon salup chopped
 1 teaspoon freshly ground pepper
 1 teaspoon sal

------_Temperature: 0.5
UP CHOPPED
 1 TEASPOON FRESHLY GROUND PEPPER
 1 TEASPOON SALt
 1 cup oreniconat (about 1 cup) unswaees
 1/4 cup sugar
 1 teaspoon salt
 1 teaspoon finely chopped and chill (1 tablespoon masher and chopped can be made 2 cups part and sugar

 1/4 cup wateranes
 1 tablespoon somes tant (corn paper
 1 1/4 teaspoons sugar
 

<ipython-input-7-ff965bbda818>:21: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


1 cup (1/2 cup water
 t (corn paper
 1 1/4 teaspoons sugar
 1 cup (1/2 cup water
 

------_Temperature: 1.0
T (CORN PAPER
 1 1/4 TEASPOONS SUGAR
 1 CUP (1/2 CUP WATER
 1 2 teaspoons minced garlic 3/4 hou)
 chopped
 kee white chicken or Tomatoes
 4 dried rise rice butter
 1 tsp
 1/2 cup wootricnic pear
 3 tablespoon some water
 12 teaspoons pepper
 1 tablespoon lemon juice
 1 tablespoon baking powder
 1 Afting or priceda
 1 capsone cloves
 or ado blackened can be grill Grilled (fengs
 1/8 teaspoon fresh lemon juice
 1 3/4 cups with green possor any white milk
 1 lemon juice
 1 3/4 cups with green possor any white milk
 1 

------_Temperature: 1.2
LEMON JUICE
 1 3/4 CUPS WITH GREEN POSSOR ANY WHITE MILK
 1 1/2 cups aamasstill
 Pinch thbu cholies
 -toughhat 

apinigmetta

Preheat tp top.
iclates from roasted corn (plus 0 12 gardorch of stir for let cream oration
 room temperatureded
 4 exces or stack lip chopped Cremeal (such as 6 R-iver
 homeras and parsley
 zuckler Philk
 4 cups grill b

3760/3760 [==============================] - 467s 124ms/step - loss: 1.0218
--- Generating with seed: "r. DO AHEAD: Can be made 4 hours ahead. Let stand at room te"

------_Temperature: 0.2
R. DO AHEAD: CAN BE MADE 4 HOURS AHEAD. LET STAND AT ROOM TEmperwaks, about 3 minutes. Add the sugar, and salt and pepper. Stir in a large saucepan over medium heat and let stand until smooth. Stir in the potatoes and set aside of chopped and cook until smooth. Stir in a large bowl over medium heat and simmer until smooth. Stir in a bowl and sauté until smooth. Stir in the sugar, and salt and pepper. Stir in a bowl over medium heat and simmer until smooth.er. Stir in a bowl over medium heat and simmer until smooth.

------_Temperature: 0.5
ER. STIR IN A BOWL OVER MEDIUM HEAT AND SIMMER UNTIL SMOOTH. Transfer to a boiling pot over medium heat and toss to combine. Cover and cook until soft peaks and cooked through, about 1 , 3 minutes and cover. Cover pancakes, salt and chill and saucevin with cooking

 1/2 teaspoon salt
 1/2 teaspoon salt
 1/2 teaspoon salt
 1/2 teaspoon salt
 1/2 teaspoon on salt
 1/2 teaspoon salt
 1/2 teaspoon salt
 1/2 teaspoon 

------_Temperature: 1.0
ON SALT
 1/2 TEASPOON SALT
 1/2 TEASPOON SALT
 1/2 TEASPOON freshly grated gilled
 2 tablespoons madepinchinady sesame seeds scallupper
 1 tsp flaky chile white with any fresh lemon fre
 1/2 teaspoon pure filletted ricottana
 warm separated arto or set petch fresh thyme
 2 slidhins
 cut into 1/2-inch pieces
 roasting pabstie phyllo 2 lard while
 2 tablespoons chopped fresh lemon festing 1/2 cup 
1 splole piping lightly potatoes curry powd set about 3 oz18g splole piping lightly potatoes curry powd set about 3 oz18g

------_Temperature: 1.2
 SPLOLE PIPING LIGHTLY POTATOES CURRY POWD SET ABOUT 3 OZ18G tenned freshly graweu, garlic claves, holestil Tape with 1/2 teaspoons oil.
Add ginger, poy and lime juice, s, cashes and heat foloves through attafe 8 5-quart skillets.
Reduce povele.) Place the zestless 3. 1/2 mediv

 2 orhime
 deep divide disker
 plus 4 - grated
 2 potaties
 for grill
 deep divide disker
 plus 4 - grated
 2 potaties
 for grill


Epoch 15
3760/3760 [==============================] - 478s 127ms/step - loss: 0.9847
--- Generating with seed: " cook green beans in same skillet over high heat, stirring o"

------_Temperature: 0.2
 COOK GREEN BEANS IN SAME SKILLET OVER HIGH HEAT, STIRRING Occasionally, until smooth. Remove from heat. Add the chicken over and cook, stirring occasionally, until golden brown, about 1 minute. Remove from heat. Add the salt and pepper. Add the pastry and cook on a baking sheet. Sprinkle with salt and pepper. Set aside.
Preheat oven to 350°F. Add the cake to a simmer in a small bowl. Add the salt and pepper. Set aside.
Place parts of the butter in a small and pepper. Set aside.
Place parts of the butter in a small

------_Temperature: 0.5
 AND PEPPER. SET ASIDE.
PLACE PARTS OF THE BUTTER IN A SMALL bowl. Place or pan sprinkle with salt.
Sprinkle with salt and 

Preheat oven to 450°F. Stir in the egg whites with salt and pepper. Bake for the second to seare seeds and preheat to 425°F. Sprinkle with salt and pepper. Cook, stirring, until sugar and fresh celery stems. Cook until browned, ab, until sugar and fresh celery stems. Cook until browned, ab

------_Temperature: 1.0
, UNTIL SUGAR AND FRESH CELERY STEMS. COOK UNTIL BROWNED, ABout 5 minutes. Season with garlic until cake, 1 1/2 teaspoon baking sheet with chicken in aleman, wine among bread for 2 Heat 1 inchant lemon, brush sdriherani with crosswo ifideed scallions, plusdhing on the bits.

Caythin with Peanuts.

Dat Hot Bring water

2 toasuan tomatoes
 olive cuti/queates
 Thinly sliced ice into ancav-saucepan ribs (the presch anirgare)
 1 (15-oy sheet in flour
 more (cusbs (the presch anirgare)
 1 (15-oy sheet in flour
 more (cus

------_Temperature: 1.2
BS (THE PRESCH ANIRGARE)
 1 (15-OY SHEET IN FLOUR
 MORE (CUS a fetach
 cranbes)
 1/2 teaspoon
 2 tablespoons fresh boquegjonal
 Drolls and